## Callin Switzer
### 23 Sept 2019
### Simulate data for training neural network 
### This uses the "one torque" or  the "under-determined" model

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
from multiprocessing import Pool, cpu_count
import multiProcTraj_one_torque as multiProcTraj # note that this is a custom-written file 

print(sys.version)

3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]


In [4]:
now = datetime.now()
print("last run on " + str(now))

# define directories
baseDir = os.getcwd()
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
dataDir = r"D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/data"

if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
    
if not os.path.exists(dataDir):
    os.mkdir(dataDir)
    
if not os.path.exists(savedModels):
    os.mkdir(savedModels)

last run on 2019-09-23 15:18:31.142186


In [5]:
# for loop to run parallel processes a few times

for ii in range(5):
    p = Pool(cpu_count()-2)
    stt = time.time()   
    bb = p.map(multiProcTraj.flyBug_firstLast, range(100000) ) 
    print(time.time() -stt)

    p.close()
    p.join()
    print(time.time() -stt)
    newDF = pd.DataFrame(bb, columns= ["x0", "xf", "xd0", "xdf", "y0", "yf", "yd0", "ydf", "theta0", \
            "thetaf", "thetad0", "thetadf", "phi0", "phif", "phid0", "phidf", "F", "alpha", "tau0"])
    
    tstamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
    print(tstamp)
    
    newDF.to_csv(os.path.join(dataDir, 
                          "RandomInitCond_" + tstamp + ".csv"),
             index = False)

59.85388231277466
60.30206656455994
2019_09_23__15_19_33
58.32891225814819
58.70071887969971
2019_09_23__15_20_34
60.39499473571777
60.744532108306885
2019_09_23__15_21_37
58.075815200805664
58.487144470214844
2019_09_23__15_22_38
61.45731234550476
61.84253168106079
2019_09_23__15_23_42
